# Factor de inflación de la varianza (VIF)

In [1]:
%pylab
%matplotlib inline

Using matplotlib backend: MacOSX
Populating the interactive namespace from numpy and matplotlib


## Importación de variables

In [2]:
import pandas as pd

wine = pd.read_csv('winequality-red.csv', sep = ';')
wine.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000
mean,8.319637,0.527821,0.270976,2.538806,0.087467,15.874922,46.467792,0.996747,3.311113,0.658149,10.422983,5.636023
std,1.741096,0.179060,0.194801,1.409928,0.047065,10.460157,32.895324,0.001887,0.154386,0.169507,1.065668,0.807569
min,4.600000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,0.990070,2.740000,0.330000,8.400000,3.000000
25%,7.100000,0.390000,0.090000,1.900000,0.070000,7.000000,22.000000,0.995600,3.210000,0.550000,9.500000,5.000000
50%,7.900000,0.520000,0.260000,2.200000,0.079000,14.000000,38.000000,0.996750,3.310000,0.620000,10.200000,6.000000
75%,9.200000,0.640000,0.420000,2.600000,0.090000,21.000000,62.000000,0.997835,3.400000,0.730000,11.100000,6.000000
max,15.900000,1.580000,1.000000,15.500000,0.611000,72.000000,289.000000,1.003690,4.010000,2.000000,14.900000,8.000000


## Obtención del valor del VIF

Vamos a calcular el valor del VIF para todas las variables menos la objetivo. Para esto se realiza una regresión lineal de cada una de las variables frente al resto y aplicamos la fórmula del VIF
$$
    VIF_i = \frac{1}{1 - R_i^2}
$$

In [3]:
from sklearn.linear_model import LinearRegression

# Borrado de la variable objetivo
wine_vif = wine.copy(deep = True)
features = list(wine_vif.columns)
features.remove('quality')
wine_vif = wine_vif[features]

for i in range(len(features)):
    var = features[i]
    fet = features[:]
    fet.remove(var)
    
    x = wine_vif[fet]
    y = wine[var]
    
    model = LinearRegression()
    model.fit(x, y)
    
    vif = 1 / (1 - model.score(x, y))
    
    print 'El valor del VIF para la variable', var, 'es:', vif

El valor del VIF para la variable fixed acidity es: 7.76751156571
El valor del VIF para la variable volatile acidity es: 1.78938968223
El valor del VIF para la variable citric acid es: 3.12802212436
El valor del VIF para la variable residual sugar es: 1.70258790815
El valor del VIF para la variable chlorides es: 1.4819323298
El valor del VIF para la variable free sulfur dioxide es: 1.96301914148
El valor del VIF para la variable total sulfur dioxide es: 2.18681278859
El valor del VIF para la variable density es: 6.34376018586
El valor del VIF para la variable pH es: 3.32973162358
El valor del VIF para la variable sulphates es: 1.42943374939
El valor del VIF para la variable alcohol es: 3.03115981988


En los resultados se puede observar que hay dos variables (`fixed acidity` y `density`) que tienen valores por encima de 5. Se ha de eliminar la que tiene el valor más alto y volve a ejecutar el proceso.

In [4]:
# Borrado de la variable objetivo
features = list(wine_vif.columns)
features.remove('fixed acidity')
wine_vif = wine_vif[features]

for i in range(len(features)):
    var = features[i]
    fet = features[:]
    fet.remove(var)
    
    x = wine_vif[fet]
    y = wine[var]
    
    model = LinearRegression()
    model.fit(x, y)
    
    vif = 1 / (1 - model.score(x, y))
    
    print 'El valor del VIF para la variable', var, 'es:', vif

El valor del VIF para la variable volatile acidity es: 1.78496308191
El valor del VIF para la variable citric acid es: 2.78055659725
El valor del VIF para la variable residual sugar es: 1.38637548021
El valor del VIF para la variable chlorides es: 1.40123158238
El valor del VIF para la variable free sulfur dioxide es: 1.93920892272
El valor del VIF para la variable total sulfur dioxide es: 2.06939644439
El valor del VIF para la variable density es: 2.43009645365
El valor del VIF para la variable pH es: 1.61077513136
El valor del VIF para la variable sulphates es: 1.39638176518
El valor del VIF para la variable alcohol es: 2.13606729055


Una vez eliminada la variable que presenta el VIF más alto el valor de este para resto de las variables cambia. Ahora podemos ver que ninguna de las variable restantes tiene un valor del VIF superior a cinco. Por lo que se ha eliminado las variables colineales de este conjunto de datos.

## Automatización del proceso de selección

In [5]:
def calculateVIF(data):
    features = list(data.columns)
    num_features = len(features)
    
    model = LinearRegression()
    
    result = pd.DataFrame(index = ['VIF'], columns = features)
    result = result.fillna(0)
    
    for ite in range(num_features):
        x_features = features[:]
        y_featue = features[ite]
        x_features.remove(y_featue)
        
        x = data[x_features]
        y = data[y_featue]
        
        model.fit(data[x_features], data[y_featue])
        
        result[y_featue] = 1/(1 - model.score(data[x_features], data[y_featue]))
    
    return result

wine_vif = wine.copy(deep = True)
features = list(wine_vif.columns)
features.remove('quality')
wine_vif = wine_vif[features]

calculateVIF(wine_vif)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
VIF,7.767512,1.78939,3.128022,1.702588,1.481932,1.963019,2.186813,6.34376,3.329732,1.429434,3.03116


In [6]:
def selectDataUsingVIF(data, max_VIF = 5):
    result = data.copy(deep = True)
    
    VIF = calculateVIF(result)
    
    while VIF.as_matrix().max() > max_VIF:
        col_max = np.where(VIF == VIF.as_matrix().max())[1][0]
        features = list(result.columns)
        features.remove(features[col_max])
        result = result[features]
        
        VIF = calculateVIF(result)
        
    return result

calculateVIF(selectDataUsingVIF(wine_vif))

,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
VIF,1.784963,2.780557,1.386375,1.401232,1.939209,2.069396,2.430096,1.610775,1.396382,2.136067
